# AR结果分组

In [10]:
import pandas as pd
import os

In [11]:
folder = r'/data/efs/mid/ar/report/'
excel_file = r'AR_Performance_two_weeks20200713.xlsx'

In [12]:
excel_path = os.path.join(folder, excel_file)

读取数据

In [13]:
if os.path.exists(csv_path):
    dfsource = pd.read_excel(excel_path, sheet_name='Sheet1', encoding='utf-8')
    print(dfsource.head())
    print(len(dfsource))
    print(dfsource.columns)

   TaskId  DocumentId  ProcessConfigId  record_dataPoint_DataPointId  \
0  282557   100444269               14                          2103   
1  216544   271330462               13                          2117   
2  282557   100444269               14                          2103   
3  282557   100444269               14                          2103   
4  282557   100444269               14                          2103   

  record_dataPoint_BusinessName record_InvestmentId  record_InvestmentType  \
0                           TER          F00000NUJK                      1   
1                           MER          F0CAN05P2H                      1   
2                           TER          F00000PDVZ                      1   
3                           TER          F00000PDW3                      1   
4                           TER          F00000PDW8                      1   

   ProviderId                                 ProviderName  \
0  0C00006EOL  PGIM India Asset Mana

构建新表

In [14]:
new_dfsource = dfsource[['TaskId', 'DocumentId', 'ProcessConfigId', 'ProviderId', 'ProviderName', 'Prospectus']]

将index作为新的序号列

In [15]:
new_dfsource.reset_index(inplace=True)

In [16]:
new_dfsource

,index,TaskId,DocumentId,ProcessConfigId,ProviderId,ProviderName,Prospectus
0,0,282557,100444269,14,0C00006EOL,PGIM India Asset Management Private Limited,FN
1,1,216544,271330462,13,0C00001CY1,Franklin Templeton Investments Corp,FN
2,2,282557,100444269,14,0C00006EOL,PGIM India Asset Management Private Limited,FN
3,3,282557,100444269,14,0C00006EOL,PGIM India Asset Management Private Limited,FN
4,4,282557,100444269,14,0C00006EOL,PGIM India Asset Management Private Limited,FN
...,...,...,...,...,...,...,...
233910,233910,298229,286799288,13,0C00002DPM,London Life Insurance Company,TN
233911,233911,298229,286799288,13,0C00002DPM,London Life Insurance Company,TN
233912,233912,298229,286799288,13,0C00002DPM,London Life Insurance Company,TN
233913,233913,298229,286799288,13,0C00002DPM,London Life Insurance Company,TN


将Prospectus进行统计

In [17]:
group_table = new_dfsource.groupby(['TaskId', 'DocumentId', 'ProcessConfigId', 'ProviderId', 'ProviderName', 'Prospectus'])['index'].count().reset_index(name='count')

In [18]:
group_table

,TaskId,DocumentId,ProcessConfigId,ProviderId,ProviderName,Prospectus,count
0,136711,243538680,14,0C00001P1X,Kotak Mahindra Asset Management Co Ltd,FN,31
1,136711,243538680,14,0C00001P1X,Kotak Mahindra Asset Management Co Ltd,TN,23
2,136711,243538680,14,0C00001P1X,Kotak Mahindra Asset Management Co Ltd,TP,308
3,136713,242584949,14,0C00008JQU,Franklin Templeton Asst Mgmt(IND)Pvt Ltd,FN,79
4,136713,242584949,14,0C00008JQU,Franklin Templeton Asst Mgmt(IND)Pvt Ltd,TN,23
...,...,...,...,...,...,...,...
2798,311738,287734974,9,0C00001M0F,Carmignac Gestion,TN,22
2799,311739,287734972,9,0C00001M0F,Carmignac Gestion,TN,28
2800,311743,287735021,9,0C00001M0F,Carmignac Gestion,TN,22
2801,311744,287734976,9,0C00001M0F,Carmignac Gestion,TN,22


行转列

In [19]:
final_table = pd.DataFrame(pd.pivot_table(group_table, index = ['TaskId', 'DocumentId', 'ProcessConfigId', 'ProviderId', 'ProviderName'],columns = 'Prospectus', values = 'count'))

In [20]:
final_table.reset_index(inplace=True)

In [21]:
final_table

Prospectus,TaskId,DocumentId,ProcessConfigId,ProviderId,ProviderName,FN,FP,TN,TP
0,136711,243538680,14,0C00001P1X,Kotak Mahindra Asset Management Co Ltd,31.0,NaN,23.0,308.0
1,136713,242584949,14,0C00008JQU,Franklin Templeton Asst Mgmt(IND)Pvt Ltd,79.0,NaN,23.0,277.0
2,136719,243188371,14,0C00009532,Edelweiss Asset Management Limited,68.0,NaN,6.0,234.0
3,136737,242122271,14,0C0000BMDA,ITI Asset Management Limited,10.0,NaN,NaN,19.0
4,141998,243188468,14,0C00008JNV,Sundaram Asset Management Company Ltd,16.0,NaN,34.0,403.0
...,...,...,...,...,...,...,...,...,...
1196,311738,287734974,9,0C00001M0F,Carmignac Gestion,NaN,NaN,22.0,NaN
1197,311739,287734972,9,0C00001M0F,Carmignac Gestion,NaN,NaN,28.0,NaN
1198,311743,287735021,9,0C00001M0F,Carmignac Gestion,NaN,NaN,22.0,NaN
1199,311744,287734976,9,0C00001M0F,Carmignac Gestion,NaN,NaN,22.0,NaN


In [22]:
final_table.fillna('0', inplace=True)
final_table[['FN', 'FP', 'TN', 'TP']] = final_table[['FN', 'FP', 'TN', 'TP']].astype(int)
final_table.dtypes

Prospectus
TaskId              int64
DocumentId          int64
ProcessConfigId     int64
ProviderId         object
ProviderName       object
FN                  int32
FP                  int32
TN                  int32
TP                  int32
dtype: object

In [23]:
final_table

Prospectus,TaskId,DocumentId,ProcessConfigId,ProviderId,ProviderName,FN,FP,TN,TP
0,136711,243538680,14,0C00001P1X,Kotak Mahindra Asset Management Co Ltd,31,0,23,308
1,136713,242584949,14,0C00008JQU,Franklin Templeton Asst Mgmt(IND)Pvt Ltd,79,0,23,277
2,136719,243188371,14,0C00009532,Edelweiss Asset Management Limited,68,0,6,234
3,136737,242122271,14,0C0000BMDA,ITI Asset Management Limited,10,0,0,19
4,141998,243188468,14,0C00008JNV,Sundaram Asset Management Company Ltd,16,0,34,403
...,...,...,...,...,...,...,...,...,...
1196,311738,287734974,9,0C00001M0F,Carmignac Gestion,0,0,22,0
1197,311739,287734972,9,0C00001M0F,Carmignac Gestion,0,0,28,0
1198,311743,287735021,9,0C00001M0F,Carmignac Gestion,0,0,22,0
1199,311744,287734976,9,0C00001M0F,Carmignac Gestion,0,0,22,0


输出为新的Excel

In [24]:
output_name_split = excel_file.split('.')
output_name = '{0}_report.xlsx'.format(output_name_split[0])
output_path = os.path.join(folder, output_name)

In [25]:
final_table.to_excel(output_path, encoding='utf-8', index=False)

In [26]:
if os.path.exists(output_path):
    result_table = pd.read_excel(output_path, sheet_name='Sheet1', encoding='utf-8')
    print(result_table.head())
    print(len(result_table))
    print(result_table.columns)
    print(result_table.dtypes)

   TaskId  DocumentId  ProcessConfigId  ProviderId  \
0  136711   243538680               14  0C00001P1X   
1  136713   242584949               14  0C00008JQU   
2  136719   243188371               14  0C00009532   
3  136737   242122271               14  0C0000BMDA   
4  141998   243188468               14  0C00008JNV   

                               ProviderName  FN  FP  TN   TP  
0    Kotak Mahindra Asset Management Co Ltd  31   0  23  308  
1  Franklin Templeton Asst Mgmt(IND)Pvt Ltd  79   0  23  277  
2        Edelweiss Asset Management Limited  68   0   6  234  
3              ITI Asset Management Limited  10   0   0   19  
4     Sundaram Asset Management Company Ltd  16   0  34  403  
1201
Index(['TaskId', 'DocumentId', 'ProcessConfigId', 'ProviderId', 'ProviderName',
       'FN', 'FP', 'TN', 'TP'],
      dtype='object')
TaskId              int64
DocumentId          int64
ProcessConfigId     int64
ProviderId         object
ProviderName       object
FN                  int64
FP